In [1]:
import pandas as pd
from eda_support_functions import *

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 1000)

In [3]:
RUNS_IDS = ["20240828_231219"]

In [4]:
parameters_df = load_metadata(
    "parameters", "/Users/iliasx/Documents/GitHub/box-office-prediction/metadata"
)
experiment_df = load_metadata(
    "metrics", "/Users/iliasx/Documents/GitHub/box-office-prediction/metadata"
)

In [5]:
experiment_df

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
0,20240802_224457,2024-08-02 23:24:31,decision_tree_classifier,multi_class_classification,full,random_search,True,none,MinMaxScaler,0.011282,0.115336,None,0.498124,0.426014,0.498124,0.452741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240828_231219,2024-08-29 03:08:14,lightgbm_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.091214,1942.437653,None,NaN,NaN,NaN,NaN,3.246506e+16,2.011322,8.447956e+07,1.801806e+08,1.047992,1.098288,0.597954,0.160422,2.186235,2.510590,NaN
2,20240802_224457,2024-08-02 23:36:14,lightgbm_regressor,regression,full,random_search,True,complex,RobustScaler,0.019995,20.893485,None,NaN,NaN,NaN,NaN,1.372573e+16,1.364065,6.213266e+07,1.171569e+08,1.022013,1.044510,0.656524,0.162289,1.472104,1.679541,NaN
3,20240731_163830,2024-07-31 16:55:05,random_forest_regressor,regression,full,random_search,True,complex,StandardScaler,0.002653,993.781275,None,NaN,NaN,NaN,NaN,2.453859e+16,1.673388,6.192644e+07,1.566480e+08,1.108347,1.228433,0.529389,0.156367,1.819095,2.066391,NaN
4,20240719_214914,2024-07-19 22:31:22,decision_tree_classifier,binary_classification,small_productions,random_search,True,none,RobustScaler,0.104955,0.044046,None,0.571429,0.653061,0.561404,0.603774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.573385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805,20240809_224515,2024-08-09 23:26:31,random_forest_regressor,regression,medium_productions,random_search,True,complex,RobustScaler,0.048205,90.286047,None,NaN,NaN,NaN,NaN,4.051588e+15,1.110755,3.805986e+07,6.365209e+07,1.003929,1.007874,0.056268,0.170819,1.192253,1.381440,NaN
3806,20240808_194937,2024-08-08 19:53:55,xgboost_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.019995,7.367819,None,NaN,NaN,NaN,NaN,3.397194e+16,2.057118,1.062804e+08,1.843148e+08,1.057773,1.118884,0.579439,0.198934,2.269904,2.718844,NaN
3807,20240802_224457,2024-08-02 23:15:24,logistic_regression,multi_class_classification,small_productions,random_search,False,none,RobustScaler,0.019995,0.156260,None,0.587189,0.491697,0.587189,0.533433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3808,20240724_210133,2024-07-24 21:20:18,decision_tree_classifier,multi_class_classification,medium_productions,random_search,False,none,StandardScaler,0.155938,0.783357,None,0.335726,0.475243,0.335726,0.381156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
experiment_df.groupby("run_id").aggregate(
    {
        "run_id": "count",
        "dataset_name": "nunique",
        "model_type": "nunique",
        "timestamp": "max",
    }
).rename(columns={"run_id": "count"}).sort_values("timestamp", ascending=False)

,count,dataset_name,model_type,timestamp
run_id,,,,
20240903_133208,1,1,1,2024-09-03 13:38:41
20240903_131915,1,1,1,2024-09-03 13:26:47
20240901_184413,1,1,1,2024-09-02 04:42:29
20240828_231219,224,4,11,2024-08-30 10:40:44
20240828_100405,1,1,1,2024-08-28 13:07:44
...,...,...,...,...
20240719_231427,12,2,6,2024-07-19 23:15:42
20240719_225631,2,2,1,2024-07-19 22:58:14
20240719_214914,272,4,11,2024-07-19 22:31:25


In [7]:
parameters_df = parameters_df[parameters_df["run_id"].isin(RUNS_IDS)]
experiment_df = experiment_df[experiment_df["run_id"].isin(RUNS_IDS)]

In [8]:
# parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'model__units', 'model__num_layers', 'model__dropout', 'epochs','variance_threshold','class_weight']].sort_values(by=['problem_type','dataset_name','has_outliers_removed'])

In [9]:
parameters_df[
    [
        "problem_type",
        "dataset_name",
        "has_outliers_removed",
        "feature_engineering",
        "model_type",
        "scaler",
        "variance_threshold",
        "class_weight",
        "C",
        "min_child_samples",
        "solver",
        "num_leaves",
        "n_estimators",
    ]
].sort_values(by=["model_type", "problem_type", "dataset_name", "has_outliers_removed"])

,problem_type,dataset_name,has_outliers_removed,feature_engineering,model_type,scaler,variance_threshold,class_weight,C,min_child_samples,solver,num_leaves,n_estimators
2281,binary_classification,full,False,none,decision_tree_classifier,StandardScaler,0.037741,None,NaN,NaN,NaN,NaN,NaN
3634,binary_classification,full,False,complex,decision_tree_classifier,StandardScaler,0.037741,None,NaN,NaN,NaN,NaN,NaN
1129,binary_classification,full,True,complex,decision_tree_classifier,MinMaxScaler,0.077083,None,NaN,NaN,NaN,NaN,NaN
2809,binary_classification,full,True,none,decision_tree_classifier,MinMaxScaler,0.077083,None,NaN,NaN,NaN,NaN,NaN
905,binary_classification,large_productions,False,none,decision_tree_classifier,RobustScaler,0.120883,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015,regression,medium_productions,True,complex,xgboost_regressor,MinMaxScaler,0.047197,NaN,NaN,NaN,NaN,NaN,1291.0
1362,regression,small_productions,False,complex,xgboost_regressor,MinMaxScaler,0.003615,NaN,NaN,NaN,NaN,NaN,210.0
1849,regression,small_productions,False,none,xgboost_regressor,RobustScaler,0.139163,NaN,NaN,NaN,NaN,NaN,1399.0
1164,regression,small_productions,True,none,xgboost_regressor,StandardScaler,0.107155,NaN,NaN,NaN,NaN,NaN,1031.0


In [10]:
parameters_df[parameters_df["model_type"] == "xgboost_regressor"][
    [
        "timestamp",
        "problem_type",
        "dataset_name",
        "has_outliers_removed",
        "feature_engineering",
        "model_type",
        "scaler",
        "variance_threshold",
        "n_estimators",
        "max_depth",
        "learning_rate",
        "num_leaves",
        "min_child_samples",
    ]
]

,timestamp,problem_type,dataset_name,has_outliers_removed,feature_engineering,model_type,scaler,variance_threshold,n_estimators,max_depth,learning_rate,num_leaves,min_child_samples
128,2024-08-29 15:33:04,regression,medium_productions,True,none,xgboost_regressor,RobustScaler,0.140072,1172.0,10.0,0.038741,NaN,NaN
326,2024-08-30 04:44:34,regression,full,False,none,xgboost_regressor,RobustScaler,0.023174,1067.0,8.0,0.015809,NaN,NaN
929,2024-08-29 02:35:52,regression,large_productions,False,complex,xgboost_regressor,RobustScaler,0.023174,1067.0,8.0,0.015809,NaN,NaN
1164,2024-08-29 05:18:36,regression,small_productions,True,none,xgboost_regressor,StandardScaler,0.107155,1031.0,7.0,0.127461,NaN,NaN
1362,2024-08-29 23:36:29,regression,small_productions,False,complex,xgboost_regressor,MinMaxScaler,0.003615,210.0,10.0,0.131392,NaN,NaN
1573,2024-08-29 20:16:55,regression,small_productions,True,complex,xgboost_regressor,StandardScaler,0.010296,269.0,3.0,0.122913,NaN,NaN
1849,2024-08-29 00:59:02,regression,small_productions,False,none,xgboost_regressor,RobustScaler,0.139163,1399.0,28.0,0.031144,NaN,NaN
2537,2024-08-30 09:56:21,regression,medium_productions,False,none,xgboost_regressor,RobustScaler,0.019082,979.0,13.0,0.171008,NaN,NaN
3015,2024-08-30 05:41:11,regression,medium_productions,True,complex,xgboost_regressor,MinMaxScaler,0.047197,1291.0,6.0,0.170783,NaN,NaN
3115,2024-08-30 04:09:59,regression,medium_productions,False,complex,xgboost_regressor,StandardScaler,0.010296,269.0,3.0,0.122913,NaN,NaN


In [11]:
parameters_df[parameters_df["model_type"] == "lightgbm_regressor"][
    [
        "timestamp",
        "problem_type",
        "dataset_name",
        "has_outliers_removed",
        "feature_engineering",
        "model_type",
        "scaler",
        "variance_threshold",
        "n_estimators",
        "max_depth",
        "learning_rate",
        "num_leaves",
        "min_child_samples",
    ]
]

,timestamp,problem_type,dataset_name,has_outliers_removed,feature_engineering,model_type,scaler,variance_threshold,n_estimators,max_depth,learning_rate,num_leaves,min_child_samples
1,2024-08-29 03:08:14,regression,large_productions,False,complex,lightgbm_regressor,RobustScaler,0.091214,1025.0,29.0,0.010333,30.0,20.0
729,2024-08-29 05:39:16,regression,small_productions,True,none,lightgbm_regressor,StandardScaler,0.096966,1484.0,9.0,0.002390,152.0,20.0
1016,2024-08-29 20:37:00,regression,small_productions,True,complex,lightgbm_regressor,MinMaxScaler,0.121192,468.0,4.0,0.142371,48.0,20.0
1258,2024-08-30 10:40:41,regression,medium_productions,False,none,lightgbm_regressor,RobustScaler,0.091214,1025.0,29.0,0.010333,30.0,20.0
1535,2024-08-30 04:36:18,regression,medium_productions,False,complex,lightgbm_regressor,RobustScaler,0.139157,1488.0,24.0,0.023695,237.0,20.0
1597,2024-08-29 15:49:24,regression,medium_productions,True,none,lightgbm_regressor,StandardScaler,0.096966,1484.0,9.0,0.002390,152.0,20.0
1635,2024-08-30 06:33:35,regression,large_productions,True,none,lightgbm_regressor,RobustScaler,0.091214,1025.0,29.0,0.010333,30.0,20.0
1967,2024-08-30 00:15:48,regression,small_productions,False,complex,lightgbm_regressor,RobustScaler,0.065192,1844.0,30.0,0.038304,163.0,20.0
2258,2024-08-30 01:06:27,regression,large_productions,True,complex,lightgbm_regressor,StandardScaler,0.010296,691.0,3.0,0.054356,235.0,20.0
2479,2024-08-30 05:34:27,regression,full,False,none,lightgbm_regressor,StandardScaler,0.022178,256.0,10.0,0.099759,30.0,20.0


In [12]:
display(experiment_df.sort_values(by="timestamp", ascending=True).tail(10))

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
270,20240828_231219,2024-08-30 09:34:18,decision_tree_classifier,binary_classification,medium_productions,random_search,False,none,StandardScaler,0.095074,0.693527,None,0.708940,0.504274,0.418440,0.457364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.623926
3704,20240828_231219,2024-08-30 09:34:30,xgboost_classifier,binary_classification,medium_productions,random_search,False,none,RobustScaler,0.079765,11.429377,None,0.738046,0.570093,0.432624,0.491935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.648665
725,20240828_231219,2024-08-30 09:54:38,lightgbm_classifier,binary_classification,medium_productions,random_search,False,none,RobustScaler,0.001390,1208.529870,None,0.752599,0.627907,0.382979,0.475771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.644431
392,20240828_231219,2024-08-30 09:54:39,dummy_regressor,regression,medium_productions,random_search,False,none,RobustScaler,0.028573,0.569815,None,NaN,NaN,NaN,NaN,1.172030e+16,52.044902,4.202770e+07,1.082603e+08,2.001341,4.005368,-0.082742,0.081081,54.183223,58.202160,NaN
2950,20240828_231219,2024-08-30 09:55:49,random_forest_regressor,regression,medium_productions,random_search,False,none,RobustScaler,0.048205,69.619151,None,NaN,NaN,NaN,NaN,8.015164e+15,3.761418,3.513564e+07,8.952745e+07,1.539599,2.370364,0.259545,0.106029,3.956504,4.299647,NaN
1231,20240828_231219,2024-08-30 09:55:49,decision_tree_regressor,regression,medium_productions,random_search,False,none,RobustScaler,0.039935,0.622153,None,NaN,NaN,NaN,NaN,7.488409e+15,3.353717,3.404412e+07,8.653559e+07,1.525108,2.325953,0.308207,0.118503,3.526835,3.947586,NaN
2537,20240828_231219,2024-08-30 09:56:21,xgboost_regressor,regression,medium_productions,random_search,False,none,RobustScaler,0.019082,32.040174,None,NaN,NaN,NaN,NaN,7.122656e+15,4.716881,3.596264e+07,8.439583e+07,1.607272,2.583323,0.341996,0.081081,4.919364,5.260857,NaN
1258,20240828_231219,2024-08-30 10:40:41,lightgbm_regressor,regression,medium_productions,random_search,False,none,RobustScaler,0.091214,2659.787192,None,NaN,NaN,NaN,NaN,6.530355e+15,4.256317,3.310677e+07,8.081061e+07,1.542095,2.378057,0.396714,0.128898,4.467542,5.066451,NaN
1148,20240828_231219,2024-08-30 10:40:42,dummy_classifier,multi_class_classification,small_productions,random_search,True,none,StandardScaler,0.040612,0.515673,None,0.335211,0.340318,0.335211,0.336683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
561,20240828_231219,2024-08-30 10:40:44,logistic_regression,multi_class_classification,small_productions,random_search,True,none,RobustScaler,0.110579,2.383390,None,0.445070,0.346083,0.445070,0.380402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
problem_types = ["regression", "binary_classification", "multi_class_classification"]
groupby_columns = ["dataset_name", "has_outliers_removed", "feature_engineering"]
metrics = {
    "regression": {"metric": "MAPE", "operation": "min"},
    "binary_classification": {"metric": "F1 Score", "operation": "max"},
    "multi_class_classification": {"metric": "F1 Score", "operation": "max"},
}
display_columns = {
    "regression": [
        "model_type",
        "scaler",
        "MAPE",
        "Threshold Probability Accuracy",
        "R2",
        "duration",
    ],
    "binary_classification": ["model_type", "scaler", "F1 Score", "duration"],
    "multi_class_classification": ["model_type", "scaler", "F1 Score", "duration"],
}

best_models = select_best_models(
    experiment_df, problem_types, groupby_columns, metrics, display_columns
)

# Access results
result_reg = best_models["regression"]
result_class = best_models["binary_classification"]
result_multi_class = best_models["multi_class_classification"]

In [14]:
print(
    result_reg.to_string(
        index=False, justify="center", float_format="%.2f", decimal="."
    )
)

problem_type    dataset_name     has_outliers_removed feature_engineering        model_type           scaler       MAPE   Threshold Probability Accuracy  R2   duration
 regression                full         False                  none            lightgbm_regressor StandardScaler  739.66               0.13              0.59  2992.93 
 regression   large_productions         False               complex             xgboost_regressor   RobustScaler    2.00               0.17              0.60   485.75 
 regression   large_productions          True               complex            lightgbm_regressor StandardScaler    0.85               0.20              0.58  1906.00 
 regression   large_productions          True                  none       random_forest_regressor   MinMaxScaler    0.92               0.19              0.55    91.04 
 regression  medium_productions         False               complex       decision_tree_regressor   MinMaxScaler    3.72               0.12              0.30   

In [15]:
print(
    result_class.to_string(
        index=False, justify="center", float_format="%.2f", decimal="."
    )
)

     problem_type        dataset_name     has_outliers_removed feature_engineering        model_type            scaler      F1 Score  duration
binary_classification               full         False               complex            lightgbm_classifier StandardScaler    0.62    2432.20 
binary_classification               full         False                  none       random_forest_classifier StandardScaler    0.60     151.43 
binary_classification               full          True               complex            lightgbm_classifier StandardScaler    0.54    1955.52 
binary_classification               full          True                  none            lightgbm_classifier StandardScaler    0.58    2319.24 
binary_classification  large_productions         False               complex       random_forest_classifier   RobustScaler    0.65     184.98 
binary_classification  large_productions         False                  none       random_forest_classifier   MinMaxScaler    0.62      72.90 

In [16]:
print(
    result_multi_class.to_string(
        index=False, justify="center", float_format="%.2f", decimal="."
    )
)

       problem_type           dataset_name     has_outliers_removed feature_engineering        model_type            scaler      F1 Score  duration
multi_class_classification               full         False               complex            lightgbm_classifier StandardScaler    0.57    7723.32 
multi_class_classification               full          True               complex             xgboost_classifier StandardScaler    0.46     601.92 
multi_class_classification               full          True                  none       random_forest_classifier   MinMaxScaler    0.45     110.95 
multi_class_classification  large_productions         False               complex       random_forest_classifier   RobustScaler    0.54     196.20 
multi_class_classification  large_productions         False                  none       random_forest_classifier   RobustScaler    0.50      76.39 
multi_class_classification  large_productions          True               complex            lightgbm_classifier

In [17]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(
    experiment_df,
    problem_type="binary_classification",
    metric="F1 Score",
    metric_agg="max",
)

In [18]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(
    experiment_df,
    problem_type="multi_class_classification",
    metric="F1 Score",
    metric_agg="max",
)

In [19]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(
    experiment_df, problem_type="regression", metric="MAPE", metric_agg="min"
)

In [20]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(
    experiment_df,
    problem_type="regression",
    metric="Threshold Probability Accuracy",
    metric_agg="max",
)

In [21]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(
    experiment_df, problem_type="regression", metric="R2", metric_agg="max"
)

In [22]:
plot_one_metric_of_different_models_per_dataset_with_plotly(
    experiment_df,
    problem_type="regression",
    metric="MAPE",
    metric_agg="min",
    benchmark_model="dummy_regressor",
)

In [23]:
plot_one_metric_of_different_models_per_dataset_with_plotly(
    experiment_df,
    problem_type="binary_classification",
    metric="F1 Score",
    metric_agg="max",
    benchmark_model="dummy_classifier",
)